# Parse downloaded pages

### Importing libraries

In [70]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import re

## Gather Information [ I Try with one anime ]

In [15]:
html_name = r"./Folder_with_page/page1/anime_1.html"
with open(html_name, "r",  encoding='utf-8') as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [240]:
attrs = ["animeTitle", "animeType", "animeNumEpisode","releaseDate","endDate"]
list_of_anime = []

In [52]:
anime_info = []
# Anime Title
anime_info.append(soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string)

# Taken Information
for h2 in soup.select('h2:has(+div)'):
    # I want the <h2> Information </h2>
    if h2.text == "Information" :
        # iter over the next 4 <div> information
        for inform in h2.find_all_next("div", attrs = {"class": "spaceit_pad"}, limit = 4):
            if inform.contents[1].string == "Type:":
                anime_info.append( inform.get_text(separator=" ", strip=True).split()[-1] )
            if inform.contents[1].string == "Episodes:":
                anime_info.append( inform.get_text(separator=" ", strip=True).split()[-1] )
            if inform.contents[1].string == "Aired:":
                anime_info.append( inform.get_text(separator=" ", strip=True).split()[1:4] )
                anime_info.append( inform.get_text(separator=" ", strip=True).split()[5:8] )
            
# scrab of the anime finish
list_of_anime.append(anime_info)

In [242]:
list_of_anime 

[['Fullmetal Alchemist: Brotherhood',
  'TV',
  '64',
  ['Apr', '5,', '2009'],
  ['Jul', '4,', '2010']]]

# Iterating over the folder

Define a function to scrab the anime (incomplite)

* anime_info: is a list that store the information of the anime in order of attributes list (the column of the dataframe)

Than the anime_info is store inside the __list_of_anime__ a list that store all the anime_info list. Than is pass to a daframe and create the tsv

In [109]:
def parse_time(date_list):
    # cheack the date is store as [Month, Day, Year]
    # there is date store as [empty] if film (end date), [?] if still airing
    if len(date_list) == 3:
        date_string = " ".join(date_list).replace(',', '')
        # %b = for abbreviate month
        new_date = datetime.strptime(date_string, '%b %d %Y')
    else:
        # return empty
        new_date = None
    return new_date

The function scrabbing_anime takes the __INFORMATION__ info of the anime + the __TITLE__ 

In [124]:
def scrabbing_anime(soup):
    anime_info = []
    # Anime Title
    # <h1 class="title-name h1_bold_none"><strong>Fullmetal Alchemist: Brotherhood</strong></h1>
    anime_info.append(str(soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string))
    # Taken Information
    # <h2>Information</h2> , there are a lot of h2 so i specify is written >>inside a div<<
    for h2 in soup.select('h2:has(+div)'):
        # I want the <h2> Information </h2> only
        if h2.text == "Information" :
            # iter over the next 4 <div> of Information, i go this way because
            # 1) i want to skip the "Status", is the 3th div
            # 2) more clear
            for inform in h2.find_all_next("div", attrs = {"class": "spaceit_pad"}, limit = 4):
                if inform.contents[1].string == "Type:":
                    anime_info.append( inform.get_text(separator=" ", strip=True).split()[-1] )
                if inform.contents[1].string == "Episodes:":
                    anime_info.append(int(inform.get_text(separator=" ", strip=True).split()[-1]) )
                if inform.contents[1].string == "Aired:":
                    anime_info.append( parse_time(inform.get_text(separator=" ", strip=True).split()[1:4] ))
                    anime_info.append( parse_time(inform.get_text(separator=" ", strip=True).split()[5:8] ))
                    
    # The scrabbing is finish, return the anime list with the info in order
    return anime_info

In [125]:
# scrabbing:
# Score, Ranked, Poplarity, Members
def scrabbing_anime2(soup, anime_info):
    # score
    score = soup.find("div", attrs = {"class": "fl-l score", "data-title": "score"})
    anime_info.append(float(score.contents[0].string))
    # I get in to the block of information I need
    tag = soup.find("div", attrs = {"class": "di-ib ml12 pl20 pt8"})
    # they are all store inside a <strong> tag + cleaning the data
    for strong_tag in tag.find_all('strong'):
        string = strong_tag.string.replace(",","")
        # elimite "#"
        s = int(re.sub(r"#", ' ', string))
        anime_info.append(int(s))
    return anime_info

In [127]:
attrs = ["animeTitle", "animeType", "animeNumEpisode","releaseDate","endDate","animeScore","animeRank","animePopularity","animeNumMembers"]
list_of_anime = []

# from page 1 to 130
# I tried only page 1
for page in tqdm(range(1,2)):
    folder = "./Folder_with_page/page"+str(page)
    for anime in os.listdir(folder):
        with open(folder + "/" + anime, "r",  encoding='utf-8') as fp:
            soup = BeautifulSoup(fp, "html.parser")
        anime_info = scrabbing_anime(soup)
        anime_info + scrabbing_anime2(soup, anime_info)
        list_of_anime.append(anime_info)
        
# Creating the DataFrame
df = pd.DataFrame(list_of_anime, columns = attrs)
# Creating the tsv file
df.to_csv('anime.tsv', index = False, sep = '\t')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.16s/it]


In [128]:
# check the type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   animeTitle       50 non-null     object        
 1   animeType        50 non-null     object        
 2   animeNumEpisode  50 non-null     int64         
 3   releaseDate      50 non-null     datetime64[ns]
 4   endDate          38 non-null     datetime64[ns]
 5   animeScore       50 non-null     float64       
 6   animeRank        50 non-null     int64         
 7   animePopularity  50 non-null     int64         
 8   animeNumMembers  50 non-null     int64         
dtypes: datetime64[ns](2), float64(1), int64(4), object(2)
memory usage: 3.6+ KB
